In [ ]:
from dataclasses import dataclass
import random
from typing import List, Tuple

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from cyslf.optimize import optimize_player_assignment
from cyslf.models import Player, Team, League
from cyslf.scorers import *
from cyslf.utils import practice_fields, max_distance

%load_ext autoreload
%autoreload 2

In [ ]:
def get_distance(p1, p2):
    return ((p2[0]-p1[0])**2 + (p2[1]-p1[1])**2)**0.5

practice_fields = [
    (42.38980588369338, -71.13301159861518), # danehy
    (42.37292538868882, -71.08361099742957), # gsm
    (42.376852476244125, -71.12084010564928), # common
]

# Find the mean location (for generating fake player data)
lats, longs = list(zip(*practice_fields))
mean_lat = np.mean(lats)
mean_long = np.mean(longs)
print(mean_lat, mean_long)

In [ ]:
# SET UP LEAGUE WITH TOY DATA
random.seed(5)

WEEKDAYS = ["M", "T", "W", "R", "F"]

players = []
for i, letter in enumerate("ABCDEFGHIJKLMNOPQ"):
    grade = random.randint(4, 5)
    skill = random.randint(1, 5) if i != 0 else 10
    location = random.gauss(mean_lat, max_distance/5), random.gauss(mean_long, max_distance/5)
    players.append(Player(
        id=i,
        first_name=letter,
        last_name=letter,
        grade=grade,
        skill=skill,
        unavailable_days=random.choice(WEEKDAYS),
        location=location,
    ))

teams=[]
for i in range(1, 6):
    location = practice_fields[(i-1) % len(practice_fields)]
    teams.append(Team(name=str(i), practice_day=WEEKDAYS[i-1], location=location))    

# ASSIGN PlAYERS TO TEAMS    
league = League(teams=teams)
for i, player in enumerate(players[:-2]):
    # Optimized assignment
    optimize_player_assignment(player, league)
    print(
        f"~~~ {score_skill(league):.2f} "
        f"{score_grade(league):.2f} "
        f"{score_size(league):.2f} "
        f"{score_convenience(league):.2f} ~~~"
    )


# SHOW RESULTS
print(league)
for team in league.teams:
    if len(team.players) == 0:
        continue
    lats, longs = list(zip(*[p.location for p in team.players]))
    plt.scatter(lats, longs, s=25, label=team.name)
    
field_lats, field_longs = list(zip(*practice_fields))
plt.scatter(field_lats, field_longs, color="k", s=50)
plt.legend()

In [ ]:
# reg_data = pd.read_csv("input - exampleRegistrationData.csv")
# print(reg_data.shape)
# reg_data.head()

In [ ]:
# parent_requests = pd.read_csv("input - exampleParentRequests.csv")
# print(parent_requests.shape)
# parent_requests.head()

In [ ]:
# player_data = reg_data.merge(
#     parent_requests,
#     how="left",
#     left_on="Lastname",
#     right_on="Player Last Name"
# )
# # Names seem borked. I'll pretend I can join on last name,
# # but IRL this doesn't work. Ideally the input is a single
# # csv with some constraints?
# player_data.head()

In [ ]:
# print(player_data.Assessment.value_counts())
# print()
# print(player_data.Grade.value_counts())

In [ ]:
# SET UP LEAGUE WITH REAL DATA
# # Set up players
# players = []
# for i, row in player_data.iterrows():
#     skill = row.Assessment
#     if pd.isna(skill) or not skill[0].isdigit():
#         continue
#         # Ignore blank, Premier, Kick Start (ask Jason later)
#     p = Player(
#         id=i,
#         first_name=row.Firstname,
#         last_name=row.Lastname,
#         grade=row.Grade,
#         skill=int(skill[0]),
#     )
#     players.append(p)
# players = sorted(players, key=lambda p: p.skill)

# # Set up teams
# teams = []
# for name in player_data["Assigned Team"].unique():
#     if pd.isnull(name):
#         continue
#     teams.append(Team(name))
# teams = sorted(teams, key=lambda t: t.name)
# print(teams)

# for i, player in enumerate(players):
#     teams = assign_player(player, teams)
#     best_team = None
#     best_score = 0
#     teams[i % len(teams)].add_player(player)
#     print(get_skill_score(teams))

In [ ]:
# def mad(x): 
#     return np.nanmedian(abs(x - np.nanmedian(x)))

# long = reg_data.Longitude.values
# lat = reg_data.Latitude.values
# plt.scatter(reg_data.Longitude, reg_data.Latitude, marker=".")

# plt.xlim(np.nanmedian(long) - 5 * mad(long), np.nanmedian(long) + 5 * mad(long))
# plt.ylim(np.nanmedian(lat) - 5 * mad(lat), np.nanmedian(lat) + 5 * mad(lat))

# for k, v in practice_field_locations.items():
#     print(v)
#     plt.scatter(v[1], v[0])